In [ ]:
%pylab inline
import glob
import pandas as pd
import os
import datetime as dt
import seaborn as sns

import behav
from behav import plotting, utils, loading
data_path = '/mnt/cube/RawData/Zog/'

subjects = (
            'B1109',
            'B1222'
            )

## Loads Data

In [ ]:
%%time
behav_data = behav.loading.load_data_pandas(subjects,data_path)

## plot performance over past two weeks in calendar

In [ ]:
for subj,data in behav_data.iteritems():
    behav.plotting.plot_filtered_performance_calendar(subj,data,num_days=20)    

##plot accuracy per stim

In [ ]:
figsize(16.0, 8.0)
for subj,df in behav_data.items():
    behav.plotting.plot_filtered_accperstim(subj, df, extract_stim_names=True)

## Plot percent correct by block with confidence bounds

In [ ]:
behav_data = behav.loading.load_data_pandas(subjects,data_path)
figsize(16.0, 16.0)

n_birds = len(behav_data.keys())
fig, ax = plt.subplots(nrows=n_birds,ncols=1)

for s, subj in enumerate(subjects):
    df = behav_data[subj]
    data_to_analyze = df[(df.response<>'none')&(df.type_=='normal')].sort()
    data_to_analyze = data_to_analyze[-10000:]
    data_to_analyze = data_to_analyze.reset_index()
    
    block_size = 100
    
    blocks = data_to_analyze.index / block_size
    blocked = data_to_analyze.groupby(blocks)
    
    sca(ax[s])
    correct = blocked['correct'].agg({'correct': lambda x: np.mean(x.astype(float))})
    correct.plot(ax=ax[s])
    
    ci = behav.utils.binomial_ci(0.5*block_size,block_size)
    axhspan(ci[0],ci[1],color='grey',alpha=0.5,axes=ax[s])
    axhline(0.5,linestyle=':',axes=ax[s],color='grey') 
    
    ylim([0,1.0])
    title(subj)
    ylabel('proportion')
    
    bias = blocked['response'].agg({'left response': lambda x: np.mean((x=='L').astype(float))})
    bias.plot(ax=ax[s],color='green')
    legend(loc=3)

##plot accuracy today

In [ ]:
figsize(16.0, 8.0)
for subj,df in behav_data.items():
    behav.plotting.plot_daily_accuracy(subj, df, x_axis='trial_num')

# Plot number of trials and feeds for past week

In [ ]:
mplib_colors = ['b','g','r', 'c', 'm', 'y']
figsize(16.0, 4.0)

for subj,df in behav_data.items():
    data_to_analyze = df[df.index>(dt.datetime.today()-dt.timedelta(weeks=1))]
    if not data_to_analyze.empty:
        blocked = data_to_analyze.groupby(lambda x: (x.date()-dt.datetime.now().date()).days)
        
        days = np.sort(blocked.groups.keys())
        trials_per_day = blocked['response'].count().values
        
        line_color = mplib_colors.pop(0)
        
        line = plot(days, trials_per_day, label=subj + ' trials per day', c=line_color)
        
        if len(days) == 1:
            plot(0, trials_per_day[-1], 'o', c=line_color)
        
        aggregated = blocked.agg({'correct': lambda x: np.mean(x.astype(float)),
                                  'response': lambda x: np.mean((x=='L').astype(float)),
                                  'type_': lambda x: np.sum((x=='normal').astype(float)),
                                  'reward': lambda x: np.sum((x==True).astype(float)),
                                    })
        
        aggregated['reward'].plot(ax=gca(), label=subj + ' feeds per day', ls='--', c=line_color)
        if len(days) == 1:
            plot(0, aggregated['reward'][0], 'o', c=line_color)

title('trials and feeds per day')
h = legend(loc='upper left')

